In [2]:
# Imports

import nltk
import requests
import time
import pandas as pd
import os
import re
import gensim
import gensim.corpora as corpora
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import streamlit as st
import torch
import numpy as np
import tensorflow as tf
import gensim.downloader as api
import tensorflow_hub as hub
import nlpaug.augmenter.word as naw
import random
import warnings
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from collections import Counter
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoTokenizer
from langchain import PromptTemplate, LLMChain
from dotenv import find_dotenv, load_dotenv
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, Word2Vec, KeyedVectors
from sklearn.manifold import TSNE
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorboard.plugins import projector
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence
from tensorboard.plugins import projector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model, Sequential
from scipy.spatial.distance import euclidean, cosine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nlpaug.util import Action
from scipy import spatial

load_dotenv()

# ---------- Functions ---------- #

# Preprocessing function
def preprocessing(text):
    # Corrected spelling on lower case text
    corrected_text = str(TextBlob(text.lower()).correct())

    return corrected_text

# ---------- Loading the dataset ---------- #

df = pd.read_csv('/Users/alexandrecogordan/Documents/ESILV/Ongoing/Machine Learning For NLP/Project 2/NLP_Data_Exploration_and_Modeling/yelp_reviews.csv')

df.drop_duplicates(inplace=True)
df.dropna(subset=['text', 'rating', 'location'], inplace=True)

# ---------- Preprocessing ---------- #

stop_words = set(stopwords.words('english'))

def review_to_vector(review, model):
    words = review.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

/Users/alexandrecogordan/miniconda3/envs/tensorflow/lib/python3.10/site-packages/tensorflow_hub/__init__.py:61: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/Users/alexandrecogordan/miniconda3/envs/tensorflow/lib/python3.10/site-packages/huggingface_hub/inference/_text_generation.py:121: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator("best_of")
/Users/alexandrecogordan/miniconda3/envs/tensorflow/lib/python3.10/site-packages/huggingface_hub/inference/_text_generation.py:140: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydan

In [13]:
def semantic_search(query, model, reviews, topn=10):
    query_vector = review_to_vector(query, model)

    # Calculate similarity between query and each review
    similarities = []

    restaurant_set = set()

    for review in reviews:
        review_vector = review_to_vector(review, model)
        similarity = 1 - spatial.distance.cosine(query_vector, review_vector)
    
        # ## 

        # Find the row in the df dataframe that corresponds to the review
        row = df[df['cleaned_text'] == review]
        restaurant_id = row['restaurant_id'].values[0]
        
        if restaurant_id not in restaurant_set:
            similarities.append((review, similarity))
            restaurant_set.add(restaurant_id)

        # ##

        # Replace with this if this doesn't work
            
        # similarities.append((review, similarity))

    # Sort reviews by similarity
    sorted_reviews = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the topn most similar reviews
    return sorted_reviews[:topn]

model = api.load('glove-twitter-50') # Change to 200 when we have the time

user_query = "Food for families and friends"
search_results = semantic_search(user_query, model, df['cleaned_text'].tolist())

In [25]:
df['text'][2]

"Been here a few times, in just recent weeks. This doesn't count visits to the rooftop bar so it's time. Time to drop my thoughts of my experiences. Here..."

In [5]:
df = pd.read_csv('/Users/alexandrecogordan/Documents/ESILV/Ongoing/Machine Learning For NLP/Project 2/NLP_Data_Exploration_and_Modeling/yelp_reviews.csv')

In [6]:
df

,restaurant_id,text,rating,location,business_name,business_price,business_review_count,business_display_address,business_display_phone,business_categories
0,GBTPC53ZrG1ZBY3DT8Mbcw,I went on a date with my partner and I got som...,5,New Orleans,Luke,$$$,5069,"['333 Saint Charles Ave', 'New Orleans, LA 701...",(504) 378-2840,"[{'alias': 'french', 'title': 'French'}, {'ali..."
1,GBTPC53ZrG1ZBY3DT8Mbcw,Went back to Luke for the first time since COV...,4,New Orleans,Luke,$$$,5069,"['333 Saint Charles Ave', 'New Orleans, LA 701...",(504) 378-2840,"[{'alias': 'french', 'title': 'French'}, {'ali..."
2,GBTPC53ZrG1ZBY3DT8Mbcw,Luke is my favorite New Orleans restaurant. I ...,5,New Orleans,Luke,$$$,5069,"['333 Saint Charles Ave', 'New Orleans, LA 701...",(504) 378-2840,"[{'alias': 'french', 'title': 'French'}, {'ali..."
3,-Tskf8WK17rb3ZfeFuRSWA,"Went here for the Bananas Foster, because of o...",5,New Orleans,Palace Café,$$$,2017,"['605 Canal St', 'New Orleans, LA 70130']",(504) 523-1661,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ..."
4,-Tskf8WK17rb3ZfeFuRSWA,Why have I not eaten here before?! I have usua...,4,New Orleans,Palace Café,$$$,2017,"['605 Canal St', 'New Orleans, LA 70130']",(504) 523-1661,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ..."
...,...,...,...,...,...,...,...,...,...,...
678,B2-pidnIEJcI4-sy97BJww,Food amazing! Service and integrity 0. Dined ...,1,Miami,LPM Restaurant & Bar,$$$$,394,"['1300 Brickell Bay Dr', 'Miami, FL 33131']",(305) 403-9133,"[{'alias': 'french', 'title': 'French'}, {'ali..."
679,B2-pidnIEJcI4-sy97BJww,Fun place celebrated my mothers birthday here-...,5,Miami,LPM Restaurant & Bar,$$$$,394,"['1300 Brickell Bay Dr', 'Miami, FL 33131']",(305) 403-9133,"[{'alias': 'french', 'title': 'French'}, {'ali..."
680,K6RzqmTJ5yaI35Bw1BAE3g,"I've dined here twice, both times for a work r...",5,Miami,Two Chef's,$$$,143,"['8287 S Dixie Hwy', 'Coral Gables, FL 33143']",(305) 663-2100,"[{'alias': 'newamerican', 'title': 'New Americ..."
681,K6RzqmTJ5yaI35Bw1BAE3g,This is two reviews at once having been there ...,4,Miami,Two Chef's,$$$,143,"['8287 S Dixie Hwy', 'Coral Gables, FL 33143']",(305) 663-2100,"[{'alias': 'newamerican', 'title': 'New Americ..."
